Carteira baseada no número de Graham sobre o valor intrínseco de uma ação, foca em pegar empresas que sejam lucrativas e baratas: empresas operando a P/L menor que 15 e com P/VPA menor que 1,5. Multiplicando 15 x 1,5 temos o número 22,5 que nos indica a fórmula de graham. Valor Intrínseco de uma ação = $\sqrt{22,5 \times \text{LPA} \times \text{VPA}}$.

Critérios:


- Ter Lucro por ação maior que zero ,isto é, a empresa não pode estar com prejuízo atualmente
- Ter Valor Patrimonial por ação positivo, ou seja, a empresa não pode ter mais passivos (obrigações a pagar) que ativos (bens ou direitos a receber)
- Ter volume médio de negociação diário de no mínimo R$250.000,00
- Ter lucro líquido medio positivo em todos os últimos 5 exercícios.


In [93]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
from IPython.display import display, HTML

def b_print(df , n=30 , clean=True): #beauty print :)
    
    # from IPython.display import display, HTML

    # if clean : # remove tickers da mesma empresa, deixando a primeria ocorrencia
    #     df['prefixo'] = df['Papel'].astype(str).str[:4]
    #     df=df.drop_duplicates(subset='prefixo', keep='first')
    #     # df=df.drop('prefixo', axis=1) 
    
    display(HTML(df.head(n).to_html(index=False)))
    df = None


In [94]:

# atualiza info fundamentlista

from DT_atualiza_settings import *
from DT_StatusInvest import SI
SI(mercado = 'Acoes' )

 ====== SI Acoes ===== 
 ====== Busca
 ====== Download 


In [120]:

#carrega a info fundamentalista num dataframe

import os
# Caminho do arquivo local
file_path = os.path.expanduser('~/GHub/Finance-playground/data/SI_Acoes.csv')
# URL para o arquivo online
file_url = 'https://raw.githubusercontent.com/BDonadelli/Finance-playground/refs/heads/main/data/SI_Acoes.csv'

# Verificar se o arquivo existe localmente
if os.path.exists(file_path):
    # Ler o arquivo local
    dados = pd.read_csv(file_path,sep=';' , decimal=',' ,thousands ='.' )
    print("Arquivo lido localmente.")
else:
    # Ler o arquivo a partir da URL
    dados = pd.read_csv(file_url,sep=';' , decimal=',' ,thousands ='.' )
    print("Arquivo lido da URL, pode não estar autalizado.")


Arquivo lido localmente.


In [96]:
dados.columns

Index(['TICKER', 'PRECO', 'DY', 'P/L', 'P/VP', 'P/ATIVOS', 'MARGEM BRUTA',
       'MARGEM EBIT', 'MARG. LIQUIDA', 'P/EBIT', 'EV/EBIT',
       'DIVIDA LIQUIDA / EBIT', 'DIV. LIQ. / PATRI.', 'PSR', 'P/CAP. GIRO',
       'P. AT CIR. LIQ.', 'LIQ. CORRENTE', 'ROE', 'ROA', 'ROIC',
       'PATRIMONIO / ATIVOS', 'PASSIVOS / ATIVOS', 'GIRO ATIVOS',
       'CAGR RECEITAS 5 ANOS', 'CAGR LUCROS 5 ANOS', ' LIQUIDEZ MEDIA DIARIA',
       ' VPA', ' LPA', ' PEG Ratio', ' VALOR DE MERCADO'],
      dtype='object')

In [97]:
cols = [
    'TICKER', 'PRECO',
    'valor intrinseco', 'Delta (%)',
    'CAGR LUCROS 5 ANOS',
    'P/L', 'P/VP',
    'ROE', 'DIVIDA LIQUIDA / EBIT',
]

In [98]:
# criterios basicos de seleção

criterios = (
        (dados[' LIQUIDEZ MEDIA DIARIA'] > 300000) &
        (dados[' LPA'] > 0) & 
        (dados[' VPA'] > 0) & 
        (dados['CAGR LUCROS 5 ANOS'] > 0)
)

# criterios com filtros extras de qualidade

criterios_extras = (
    (criterios) & 
    (dados['ROE'] > 0.08) &
    (dados['DIVIDA LIQUIDA / EBIT'] < 4)
)

# 'DIVIDA LIQUIDA / EBIT' é NaN para bancos

bancos = [
    "ABCB4",
    "BAZA3", "BBAS3", "BBDC3", "BBDC4", "BEES3", "BEES4", "BGIP3", "BGIP4",
    "BGIP4", "BMEB3", "BMEB4", "BMGB4", "BNBR3", "BPAC11", "BPAC3", "BPAC5", "BPAN11", "BPAN4", "BPAR3",
    "BPAT33", "BRSR3", "BRSR5", "BRSR6", "BSLI3", "BSLI4", "BSLI4", "BRIV3", "BRIV4", "BMIN3", "BMIN4"
    "IRBR3", "ITUB3", "ITUB4", 
    "MERC3", "MERC4", "MODL3",  "MODL4", "MODL11",  "PINE3","PINE4", "SANB11", "SANB3","SANB4",
    "BBSE3", "CRIV3", "CRIV4", "CSAB3", "CSAB4", "FIGE3", "FIGE4", "FIGE3", "FNCN3", "IDVL3", "IDVL4" , "IRBR3",
    "BMIN3" , "BMIN4", "BIDI3", "BIDI4", "BIDI11", 
    ]

# Substitui por 0 quando o ticker for de banco
dados.loc[dados["TICKER"].isin(bancos),    "DIVIDA LIQUIDA / EBIT"] = 0


In [ ]:

# retira linha de ticker que não são mais negociados

delisted = [
    'SOMA3','JBSS3','OIBR3'
]
dados = dados[~dados['TICKER'].isin(delisted)]


In [100]:
# valor intrinseco da acao
dados['valor intrinseco'] = np.round(np.sqrt(22.5 * dados[' LPA'] * dados[' VPA']),2)
# potencial de crescimento 
dados['Delta (%)'] = np.round((dados['valor intrinseco'] / dados['PRECO'] -1)*100,2)
'''
rank() atribui uma posição relativa a cada valor da série. O menor valor recebe rank 1 (ascending=True)
Todos os valores empatados recebem o menor rank possível dentro do empate (method="min")
'''
dados["Rank"] = dados['Delta (%)'].rank(ascending=True, method="min")
# ordena por rank em ordem decrescente
dados.sort_values(by="Rank", ascending=False, inplace=True)

# ajeitando uns numeros

dados['VAL DE MERCADO (em Bilhoes)'] = dados[' VALOR DE MERCADO']  / 1e9
dados['LIQ MEDIA DIARIA (em Milhoes)'] = dados[' LIQUIDEZ MEDIA DIARIA'] / 1e6


In [101]:
b_print(dados.loc[criterios, cols],n=10)

TICKER,PRECO,valor intrinseco,Delta (%),CAGR LUCROS 5 ANOS,P/L,P/VP,ROE,DIVIDA LIQUIDA / EBIT
LIGT3,"4,81","45,73","850,73","11,57","0,78","0,32","40,83","5,10"
ALLD3,"7,47","38,60","416,73","28,55","2,01","0,42","20,88","-0,32"
BAZA3,"86,64","236,60","173,08","31,27","4,52","0,67","14,74","0,00"
COGN3,"3,46","9,43","172,54","41,43","5,36","0,56","10,43","3,26"
TASA4,"5,07","13,64","169,03","23,30","5,71","0,55","9,56","3,87"
RECV3,"10,85","27,17","150,41","57,65","5,14","0,70","13,59","1,75"
MTRE3,"4,09","9,84","140,59","8,93","8,83","0,44","4,95","5,80"
BRSR6,"18,45","43,58","136,21","0,02","5,94","0,68","11,45","0,00"
AZZA3,"24,48","57,62","135,38","35,96","6,71","0,61","9,02","3,38"
LOGG3,"28,17","65,49","132,48","33,60","6,45","0,65","10,02","3,28"


In [102]:
b_print(dados.loc[criterios_extras, cols],n=10)

TICKER,PRECO,valor intrinseco,Delta (%),CAGR LUCROS 5 ANOS,P/L,P/VP,ROE,DIVIDA LIQUIDA / EBIT
ALLD3,"7,47","38,60","416,73","28,55","2,01","0,42","20,88","-0,32"
COGN3,"3,46","9,43","172,54","41,43","5,36","0,56","10,43","3,26"
TASA4,"5,07","13,64","169,03","23,30","5,71","0,55","9,56","3,87"
RECV3,"10,85","27,17","150,41","57,65","5,14","0,70","13,59","1,75"
AZZA3,"24,48","57,62","135,38","35,96","6,71","0,61","9,02","3,38"
LOGG3,"28,17","65,49","132,48","33,60","6,45","0,65","10,02","3,28"
EUCA4,"20,29","46,61","129,72","35,10","6,36","0,67","10,54","0,99"
MELK3,"3,94","8,26","109,64","21,01","6,89","0,74","10,75","1,29"
CASH3,"3,50","7,21","106,00","29,42","7,26","0,73","10,03","-0,87"
JHSF3,"9,65","19,78","104,97","32,04","5,10","1,05","20,57","1,94"


In [103]:

# # remove empresas repetidas, mantem primeira ocorrencia

# dados['prefixo'] = dados['TICKER'].str[:4]
# # dados.drop_duplicates(subset='prefixo', keep='first')
# dados_limpo = dados.drop_duplicates(subset='prefixo', keep='first')
# # df_limpo = df_limpo.drop('prefixo', axis=1) 
# b_print(dados_limpo[colunas_exibidas])

In [105]:

# Filtro de valor (Graham + margem)
value_filter = dados['Delta (%)'] >= 25
# Catalisador de valuation — P/L deprimido
catalyst_pl = dados['P/L'] < 12
# Catalisador patrimonial — P/VPA baixo
catalyst_pvp = dados['P/VP'] < 1.2
# Pelo menos 1 catalisador ativo
dados['Catalisador'] = (
    catalyst_pl |
    catalyst_pvp
)

# Screening final 

screening = dados[
    criterios_extras &
    value_filter &
    dados['Catalisador']
].copy()
# Score de position trade (simples e robusto)
screening['Score'] = (
    (screening['Delta (%)'] / 100) * 0.5 +
    (screening['CAGR LUCROS 5 ANOS'].clip(0, 20) / 100) * 0.3 +
    ((12 - screening['P/L']).clip(0, 12) / 12) * 0.2
)
#  Ordenação final
screening.sort_values('Score', ascending=False, inplace=True)
# screening.reset_index(drop=True, inplace=True)
# screening.index = screening.index + 1


cols = [
    'TICKER', 'PRECO',
    'valor intrinseco', 'Delta (%)',
    'CAGR LUCROS 5 ANOS',
    'P/L', 'P/VP',
    'ROE', 'DIVIDA LIQUIDA / EBIT',
    'Score'
]

b_print(screening[cols],n=10)

TICKER,PRECO,valor intrinseco,Delta (%),CAGR LUCROS 5 ANOS,P/L,P/VP,ROE,DIVIDA LIQUIDA / EBIT,Score
ALLD3,"7,47","38,60","416,73","28,55","2,01","0,42","20,88","-0,32","2,31"
COGN3,"3,46","9,43","172,54","41,43","5,36","0,56","10,43","3,26","1,03"
TASA4,"5,07","13,64","169,03","23,30","5,71","0,55","9,56","3,87","1,01"
RECV3,"10,85","27,17","150,41","57,65","5,14","0,70","13,59","1,75","0,93"
AZZA3,"24,48","57,62","135,38","35,96","6,71","0,61","9,02","3,38","0,83"
LOGG3,"28,17","65,49","132,48","33,60","6,45","0,65","10,02","3,28","0,81"
EUCA4,"20,29","46,61","129,72","35,10","6,36","0,67","10,54","0,99","0,80"
JHSF3,"9,65","19,78","104,97","32,04","5,10","1,05","20,57","1,94","0,70"
MELK3,"3,94","8,26","109,64","21,01","6,89","0,74","10,75","1,29","0,69"
DEXP3,"7,61","15,55","104,34","44,31","5,99","0,90","15,00","-0,35","0,68"


Interpretação prática (position trade)

Delta alto + Score alto
→ desconto real com gatilho

Score alto, Delta médio
→ re-rating provável

Delta alto, Score baixo
→ possível value trap

In [106]:
pd.options.display.float_format = (
    lambda x: f"{x:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")
)

carteira = (screening['TICKER'].head(10)+'.SA').to_list()

import yfinance as yf
price = yf.download(
    tickers=carteira,
    period='1d',
    interval='1d',
    group_by='ticker',
    auto_adjust=True,
    progress=False
)

# Extrair preço de fechamento
precos = {}

for t in carteira:
    try:
        precos[t.replace('.SA', '')] = price[t]['Close'].iloc[-1]
    except KeyError:
        precos[t.replace('.SA', '')] = None

precos = pd.DataFrame.from_dict(precos, orient="index", columns=["Close"])


precos


,Close
ALLD3,"7,55"
COGN3,"3,51"
TASA4,"5,09"
RECV3,"11,19"
AZZA3,"24,66"
LOGG3,"27,13"
EUCA4,"21,75"
JHSF3,"9,64"
MELK3,"3,93"
DEXP3,"7,65"


In [107]:
# criterios minimos

carteira = (dados.loc[criterios]['TICKER'].head(10)+'.SA').to_list()

price = yf.download(
    tickers=carteira,
    period='1d',
    interval='1d',
    group_by='ticker',
    auto_adjust=True,
    progress=False
)

# Extrair preço de fechamento
precos = {}

for t in carteira:
    try:
        precos[t.replace('.SA', '')] = price[t]['Close'].iloc[-1]
    except KeyError:
        precos[t.replace('.SA', '')] = None

precos = pd.DataFrame.from_dict(precos, orient="index", columns=["Close"])
precos


,Close
LIGT3,"4,76"
ALLD3,"7,55"
BAZA3,"85,91"
COGN3,"3,51"
TASA4,"5,09"
RECV3,"11,19"
MTRE3,"4,20"
BRSR6,"18,33"
AZZA3,"24,66"
LOGG3,"27,11"


================================================================================================

### outra estória de porque 22,5

O número 22,5 na Fórmula de Graham é um fator de ponderação que tem um propósito específico. Essa constante foi escolhida por Graham pra ajustar a avaliação do preço justo de uma ação com base na taxa de crescimento anual esperada da empresa.

O número 22,5 é o resultado da multiplicação de 8,5 por 2,65 (8,5 x 2,65 = 22,5). O número 8,5 é a base que Graham considerou razoável pra uma empresa com taxa de crescimento zero, ou seja, uma empresa que não cresce. Já o número 2,65 representa a média do retorno exigido pelos investidores no mercado de ações durante a época de Graham, que era de aproximadamente 4,4% acima da taxa de retorno dos títulos do Tesouro dos Estados Unidos. O fator 22,5 ajuda a ajustar o preço justo com base no crescimento da empresa e na expectativa de retorno dos investidores. Esse ajuste garante que a Fórmula de Graham considere a taxa de crescimento anual esperada e reflita uma avaliação mais realista do preço justo de uma ação.

In [133]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
pd.set_option('display.max_columns', None)


url1 = "https://www.fundamentus.com.br/resultado.php"

header = {
    "User-Agent": "Mozilla/5.0",
    "X-Requested-With": "XMLHttpRequest"
}
#Verificar status HTTP
r1 = requests.get(url1, headers=header, timeout=30)
r1.raise_for_status()

dados = pd.read_html(
    StringIO(r1.text),
    decimal=",",
    thousands="."
)[0]

# limpar colunas
dados.columns = (
    dados.columns
    .str.strip()
    .str.replace(".", "", regex=False)
)

dados['ROE'] = dados['ROE'].str.replace('%', '', regex=False).str.replace(".", "", regex=False).str.replace(',', '.', regex=False).astype('float')
dados['Cresc Rec5a'] = dados['Cresc Rec5a'].str.replace('%', '', regex=False).str.replace(".", "", regex=False).str.replace(',', '.', regex=False).astype('float')
dados.rename(columns={'ROE':'ROE(%)' , 'Cresc Rec5a' : 'Cresc Rec5a(%)'},inplace=True)

# LPA e VPA
dados["LPA"] = np.where(dados["P/L"] > 0, dados["Cotação"] / dados["P/L"], np.nan)
dados["VPA"] = np.where(dados["P/VP"] > 0, dados["Cotação"] / dados["P/VP"], np.nan)

# Graham seguro
graham_base = 22.5 * dados["LPA"] * dados["VPA"]
graham_base = graham_base.where(graham_base > 0)
dados["valor intrinseco"] = np.round(np.sqrt(graham_base), 2)

# Delta
dados["Delta (%)"] = np.round(
    (dados["valor intrinseco"] / dados["Cotação"] - 1) * 100,
    2
)

# ordenar
dados = dados.sort_values("Delta (%)", ascending=False)

# print(dados [['Papel' , 'Cotação' , "Delta (%)" , "valor intrinseco" ]])

criterios = (
        (dados['Liq2meses'] > 1000000) &
        (dados['LPA'] > 0) & 
        (dados['VPA'] > 0) & 
        (dados['Cresc Rec5a(%)'] > 0)& 
        (dados['ROE(%)'] > 8) &
        (dados['Liq Corr'] > 1 )& 
        (dados['DívBrut/ Patrim'] < 1 )
)





In [135]:


print(dados[criterios][ [
    'Papel', 'Cotação',
    'valor intrinseco', 'Delta (%)',
    'P/L', 'P/VP',
    'ROE(%)'
]].head(20) )

      Papel  Cotação  valor intrinseco  Delta (%)  P/L  P/VP  ROE(%)
505   COGN3     3,46              9,47     173,70 5,36  0,56   10,43
486   VTRU3    15,09             40,92     171,17 4,31  0,71   16,58
469   RIAA3    10,37             26,62     156,70 3,52  0,97   27,56
502   RECV3    10,85             27,13     150,05 5,14  0,70   13,59
555   AZZA3    24,48             57,40     134,48 6,71  0,61    9,02
559   LOGG3    28,17             60,89     116,15 6,98  0,69    9,83
563   CASH3     3,50              7,21     106,00 7,26  0,73   10,03
521   DEXP3     7,61             15,55     104,34 5,99  0,90   15,00
573   MELK3     3,94              7,88     100,00 7,60  0,74    9,75
552   ISAE4    29,60             56,37      90,44 6,67  0,93   13,99
520   GRND3     4,78              9,04      89,12 5,94  1,06   17,92
587   EZTC3    15,25             28,29      85,51 7,88  0,83   10,60
532   SAPR4     8,73             15,87      81,79 6,19  1,10   17,71
644   PFRM3     8,50             1